# 如何给 agent 正确的增加 memory

1. 将 memory 插入到提示词模板中

In [1]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

llm = ChatDeepSeek(
    model="deepseek-chat",
    api_key=openai_api_key,
    api_base=openai_api_base,
    temperature=0.1,
)


In [ ]:
#构建一个搜索工具
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain(
    llm=llm,
    verbose=True
)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

In [ ]:
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
)

# 查看默认 prompt
print(agent_chain.agent.prompt.messages)

# 目前还不知道
agent_chain.run("我叫什么名字？")

In [ ]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    # 设置参数，插入历史聊天
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),
                                 MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory #记忆组件
    )

# 查看新的 prompt
print(agent_chain.agent.prompt.messages)

In [ ]:
agent_chain.run("好厉害，刚才我们都聊了什么？")